# Taller en clase - Grid search vs Randomized search

## Caso de estudio

Usaremos los datos de costos de una compañia de seguros médicos. El dataset cuenta con información acerca de los asegurados (edad, sexo, bmi, hijos, fuma, región) y los costos que tuvo para la compañia esa persona.

Algunos costos en los que incurre la compañia de seguros pueden estar asociados a eventos fortuitos, pero se espera que ciertas poblaciones (personas de más edad o fumadores por ejemplo) impliquen mayores costos para las compañias.

Teniendo en cuenta los datos históricos de la compañia de seguros médicos, La compañia quiere estimar el costo de pacientes nuevos para poder ajustar mejor su esquema de costos y financiación.


### Créditos

El dataset de costos médicos es un dataset sintético creado para el libro Machine Learning with R (2nd. ed.) Brett Lantz. Packt Publishing.


In [1]:
import pandas as pd

In [2]:
# Creación de objeto pandas dataframe
patients_df = pd.read_csv('https://github.com/stedy/Machine-Learning-with-R-datasets/blob/master/insurance.csv?raw=true')

### Procesamiento

In [3]:
# Manejo de variables categoricas
# Forma 1: Reemplazo manual

patients_df_1 = patients_df.replace({'sex': {'male': 0, 'female': 1}, 
                                     'smoker': {'yes': 0, 'no': 1}, 
                                     'region': {'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3}})
patients_df_1.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7124\1282938933.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  patients_df_1 = patients_df.replace({'sex': {'male': 0, 'female': 1},


,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,0,0,16884.92400
1,18,0,33.770,1,1,1,1725.55230
2,28,0,33.000,3,1,1,4449.46200
3,33,0,22.705,0,1,2,21984.47061
4,32,0,28.880,0,1,2,3866.85520


### Partición de los datos

In [4]:
# Forma 1: Partición manual

# Uso 70% para entrenamiento (random split)
train_df = patients_df_1.sample(frac = 0.7, random_state = 42)

rest_df = patients_df_1.drop(train_df.index)

# Uso 15% para validacion y 15% para test
val_df = rest_df.sample(frac = 0.5, random_state = 42)

test_df = rest_df.drop(val_df.index)

print('train: ', train_df.shape)
print('val: ', val_df.shape)
print('test: ', test_df.shape)

train:  (937, 7)
val:  (200, 7)
test:  (201, 7)


### Entrenamiento del modelo

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor()

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 8],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(model, param_grid)
grid_search.fit(train_df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']], train_df[['charges']])

print(f'Mejores hiperparámetros: {grid_search.best_params_}')

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: Dat

Mejores hiperparámetros: {'max_depth': 4, 'min_samples_split': 10, 'n_estimators': 100}


In [6]:
from sklearn.model_selection import RandomizedSearchCV

model = RandomForestRegressor()

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 8],
    'min_samples_split': [2, 5, 10]
}

randomized_search = RandomizedSearchCV(model, param_grid, n_iter=10)
randomized_search.fit(train_df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']], train_df[['charges']])

print(f'Mejores hiperparámetros: {randomized_search.best_params_}')

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: Dat

Mejores hiperparámetros: {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': 4}


### Validación del modelo

In [7]:
from sklearn.metrics import mean_squared_error, r2_score

best_grid_search_model = grid_search.best_estimator_

charges_predicted = best_grid_search_model.predict(val_df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']])
print('mean_squared_error:', mean_squared_error(val_df['charges'], charges_predicted))
print('r2_score:', r2_score(val_df['charges'], charges_predicted))

mean_squared_error: 26541196.845326427
r2_score: 0.8203075411278447


In [8]:
best_randomized_search_model = randomized_search.best_estimator_

charges_predicted = best_randomized_search_model.predict(val_df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']])
print('mean_squared_error:', mean_squared_error(val_df['charges'], charges_predicted))
print('r2_score:', r2_score(val_df['charges'], charges_predicted))

mean_squared_error: 26361116.587509535
r2_score: 0.8215267425267125


### ¿Cómo guardar un modelo?

In [9]:
import pickle

In [10]:
with open('model.pkl', 'wb') as f:
    pickle.dump(best_randomized_search_model, f)